# Testing is good

Testing is one of the most important things we can do in our infrastructure to make sure that things are configured the way we expect them to be and that the system as a whole is operating the way we want it to.

In a world of dynamic protocols that are **designed** to continue to operate in the face of multiple failures, it's always good to make sure that you know when the system has gone through a failure.  

If routing works the way it's supposed to, you may not even be aware you have a failure until the last bandaid finally falls off and you have a total meltdown.

More importantly, Testing can be used to help gain confidence in your changes, not just for you, but for your peers, managers, and the business who depends on the network to get things done.

We're going to start as usual by grabbing all the imports we need. 

** *Note: I'm going to fly through some of these steps as I've covered them pretty thouroughly in previous blogs, please feel free to ask/comment if there's something that you'd like me to explain in further detail* **

In [26]:
from pyhpeimc.auth import *
from pyhpeimc.plat.icc import *
from pyhpeimc.plat.device import *
import jtextfsm as textfsm
import yaml
#import githubuser
import mygithub   #file not in github repo
auth = IMCAuth("http://", "10.101.0.203", "8080", "admin", "admin")

First we've got to grab the devID of the switch we wish to test 

In [27]:
devid = get_dev_details('10.101.0.221', auth.creds, auth.url)['id']
devid

'10'

Now we need to create the list of commands that we want to gather their output. For this example, we want to make sure that OSPF, as a system, is still working. So we want to gather the **display ospf peer** output so that we can tkae a look at the peers and make sure that all expected peers are still present and in a **Full/BDR** state.

In [28]:
cmd_list = ['display ospf peer']

Now that we've got the command list, we're going to use the **run_dev_cmd** function from the **pyhpeimc** library to gather this for the devid of the switch we specified above.

We'll also take a quick look at the contents of the **contents** key of the object that is returned by the run_dev_cmd function.

In [29]:
raw_text_data = run_dev_cmd(devid, cmd_list, auth.creds, auth.url)
raw_text_data['content']

'\t          OSPF Process 1 with Router ID 10.101.0.221\r\n                        Neighbor Brief Information\r\n\r\n Area: 0.0.0.0\r\n Router ID       Address         Pri  Dead-Time  Interface       State\r\n 10.101.16.1     10.101.0.1      1    36         Vlan1           Full/BDR\r\n 10.101.16.1     10.101.15.1     1    32         Vlan15          Full/BDR\r\n 10.20.1.1       10.20.1.1       1    37         GE2/0/23        Full/BDR'

Just to be sure, we'll print this out to make sure that this is the response we would actually expect for this command from that specific device OS.

In [30]:
print (raw_text_data['content'])

	          OSPF Process 1 with Router ID 10.101.0.221
                        Neighbor Brief Information

 Area: 0.0.0.0
 Router ID       Address         Pri  Dead-Time  Interface       State
 10.101.16.1     10.101.0.1      1    36         Vlan1           Full/BDR
 10.101.16.1     10.101.15.1     1    32         Vlan15          Full/BDR
 10.20.1.1       10.20.1.1       1    37         GE2/0/23        Full/BDR


We will now run this through a TextFSM template to transform this string into some structured text which will make it much easier to deal with.  

In [31]:
template = open("./Textfsm/Templates/displayospf.textfsm")
re_table = textfsm.TextFSM(template)
fsm_results = re_table.ParseText(raw_text_data['content'])
ospf_peer = [ { 'area': i[0], 'router_id' :i[1], 'address':i[2], 'pri' :i[3], 'deadtime': i[4], 'interface': i[5], 'state': i[6]} for i in fsm_results]
print ( "There are currently " + str(len(ospf_peer)) + ' OSPF peers on this device')
ospf_peer[0]

There are currently 4 OSPF peers on this device


{'address': '10.101.0.1',
 'area': '0.0.0.0',
 'deadtime': '36',
 'interface': 'Vlan1',
 'pri': '1',
 'router_id': '10.101.16.1',
 'state': 'Full/BDR'}

Now that we've got an object with all the OSPF Peers in them, let's write some quick code to see if the one specific peer, 10.20.1.1, is present in the OSPF peer table and if it's current state is Full/BDR.  This will let us know that the OSPF peer we expect to be in the table is, in fact, still in the table and in the FULL/BDR state which tells us there's a pretty good chance it's passing traffic.

I've also added an **else** clause to 

In [32]:
for peer in ospf_peer:
    if (peer['address']) == '10.20.1.1' and peer['state'] == "Full/BDR":
        print ( peer['address'] + " was the peer I was looking for and it's Full")
    else:
        print (peer['address'] + ' was not the peer I was looking for')

10.101.0.1 was not the peer I was looking for
10.101.15.1 was not the peer I was looking for
10.20.1.1 was the peer I was looking for and it's Full
 was not the peer I was looking for


# Checking IP Routes

What about checking the routing table of a remote peer?


In [33]:
cmd_list = ['display ip routing-table']

In [34]:
raw_text_data = run_dev_cmd(devid, cmd_list, auth.creds, auth.url)
raw_text_data['content']

'Routing Tables: Public\r\n\tDestinations : 58\tRoutes : 59\r\n\r\nDestination/Mask    Proto  Pre  Cost         NextHop         Interface\r\n\r\n0.0.0.0/0           Static 60   0            10.101.0.1      Vlan1\r\n10.3.10.0/24        Direct 0    0            10.3.10.1       Vlan20\r\n10.3.10.1/32        Direct 0    0            127.0.0.1       InLoop0\r\n10.10.3.0/24        Direct 0    0            10.10.3.1       Vlan3\r\n10.10.3.1/32        Direct 0    0            127.0.0.1       InLoop0\r\n10.10.10.0/24       Direct 0    0            10.10.10.1      Vlan10\r\n10.10.10.1/32       Direct 0    0            127.0.0.1       InLoop0\r\n10.10.11.0/24       Direct 0    0            10.10.11.1      Vlan11\r\n10.10.11.1/32       Direct 0    0            127.0.0.1       InLoop0\r\n10.10.12.0/24       Direct 0    0            10.10.12.1      Vlan12\r\n10.10.12.1/32       Direct 0    0            127.0.0.1       InLoop0\r\n10.10.13.0/24       Direct 0    0            10.10.13.1      Vlan13\r\n

In [35]:
print (raw_text_data['content'])

Routing Tables: Public
	Destinations : 58	Routes : 59

Destination/Mask    Proto  Pre  Cost         NextHop         Interface

0.0.0.0/0           Static 60   0            10.101.0.1      Vlan1
10.3.10.0/24        Direct 0    0            10.3.10.1       Vlan20
10.3.10.1/32        Direct 0    0            127.0.0.1       InLoop0
10.10.3.0/24        Direct 0    0            10.10.3.1       Vlan3
10.10.3.1/32        Direct 0    0            127.0.0.1       InLoop0
10.10.10.0/24       Direct 0    0            10.10.10.1      Vlan10
10.10.10.1/32       Direct 0    0            127.0.0.1       InLoop0
10.10.11.0/24       Direct 0    0            10.10.11.1      Vlan11
10.10.11.1/32       Direct 0    0            127.0.0.1       InLoop0
10.10.12.0/24       Direct 0    0            10.10.12.1      Vlan12
10.10.12.1/32       Direct 0    0            127.0.0.1       InLoop0
10.10.13.0/24       Direct 0    0            10.10.13.1      Vlan13
10.10.13.1/32       Direct 0    0            127.0.0.1

In [36]:
template = open("./Textfsm/Templates/displayiproutingtable.textfsm")
re_table = textfsm.TextFSM(template)
fsm_results = re_table.ParseText(raw_text_data['content'])
ip_routes = [ { 'DestinationMask': i[0], 'Proto' :i[1], 'Pre':i[2], 'Cost' :i[3], 'NextHop': i[4], 'Interface': i[5]} for i in fsm_results]
ip_routes[0]

{'Cost': '0',
 'DestinationMask': '0.0.0.0/0 ',
 'Interface': 'Vlan1',
 'NextHop': '10.101.0.1 ',
 'Pre': '60',
 'Proto': 'Static'}

In [37]:
for route in ip_routes:
    if route['DestinationMask'] == "10.20.10.0/24":
        print (json.dumps(route, indent =4))


{
    "Proto": "Direct",
    "Interface": "GE1/0/22",
    "NextHop": "10.20.10.1 ",
    "Pre": "0",
    "DestinationMask": "10.20.10.0/24",
    "Cost": "0"
}


# Checking VLANs

In [20]:
devid = get_dev_details('10.20.10.10', auth.creds, auth.url)['id']
devid

'349'

In [21]:
cmd_list = ['display vlan brief']

In [22]:
raw_text_data = run_dev_cmd(devid, cmd_list, auth.creds, auth.url)
raw_text_data['content']

'Brief information about all VLANs:\r\r\nSupported Minimum VLAN ID: 1\r\r\nSupported Maximum VLAN ID: 4094\r\r\nDefault VLAN ID: 1\r\r\nVLAN ID   Name                             Port\r\r\n1         default                          FGE1/0/49  FGE1/0/50  FGE1/0/51  \r\r\n                                           FGE1/0/52  XGE1/0/1  XGE1/0/2  \r\r\n                                           XGE1/0/3  XGE1/0/4  XGE1/0/5  \r\r\n                                           XGE1/0/6  XGE1/0/7  XGE1/0/8  \r\r\n                                           XGE1/0/9  XGE1/0/10  XGE1/0/11  \r\r\n                                           XGE1/0/12  XGE1/0/13  XGE1/0/14  \r\r\n                                           XGE1/0/15  XGE1/0/16  XGE1/0/17  \r\r\n                                           XGE1/0/18  XGE1/0/19  XGE1/0/20  \r\r\n                                           XGE1/0/21  XGE1/0/22  XGE1/0/23  \r\r\n                                           XGE1/0/24  XGE1/0/25  XGE1/0/26  \r\r\n

In [23]:
print (raw_text_data['content'])
























5         DoesntBelong                     


In [24]:
template = open("./TextFSM/Templates/displayvlanbrief.textfsm")
re_table = textfsm.TextFSM(template)
fsm_results = re_table.ParseText(raw_text_data['content'])

In [25]:
fsm_results
dev_vlans = [ {'vlanId': i[0], 'vlanName' : i[1]} for i in fsm_results]
dev_vlans

[{'vlanId': '1', 'vlanName': 'default'},
 {'vlanId': '5', 'vlanName': 'DoesntBelong'}]

#  Checking our work

Now that we've captured the VLANs present on the device, we can easily compare this back to the GITHUB YAML file where we originally defined what VLANs should be on the device.

First we'll create the git_vlans object from the file vlans.yaml directly from GITHUB. 

In [288]:
gitauth = mygithub.gitcreds() #you didn't think I was going to give you my password did you?
git_vlans = yaml.load(requests.get('https://raw.githubusercontent.com/netmanchris/Jinja2-Network-Configurations-Scripts/master/vlans.yaml', auth=gitauth).text)

## Cleaning up a bit

If we take a look at the gitvlans variable, we can see it's a little too deep for what we want to do. 
We're going to perform two transformations on the data here to get it to where we want it to be
- grab the contents of the git_vlans['vlans'] key which is just the list of vlans.
- use the .pop()  method on each of the vlans to get rid of the **vlanStatus** key which we don't want here

For the 

In [235]:
git_vlans = git_vlans['vlans']
for vlan in git_vlans:
    vlan.pop('vlanStatus')
git_vlans

[{'vlanId': '1', 'vlanName': 'default'},
 {'vlanId': '2', 'vlanName': 'TenantABC'},
 {'vlanId': '3', 'vlanName': 'management'},
 {'vlanId': '10', 'vlanName': 'mgmt'}]

## Comparing git_vlans and dev_vlans

Now that we've got two different list which contain a vlan dictionary for each VLAN with the exact same keys, we can do a boolean magic to see if each of the VLANs are present in the other objects. 

We'll first do this by comparing to see if all of the VLANs from the **git_vlans** object **are** actually on the device. The git_vlans objects was loaded from a YAML file on github where we defined what VLANS **should** be on the device, remember?

In [236]:
for vlan in git_vlans:
        if vlan in dev_vlans:
            print (vlan['vlanId'] + " is there")
        elif vlan not in dev_vlans:
            print (devv['vlanId'] + " is not there")

1 is there
2 is there
3 is there
10 is there


## Comparing dev_vlans to git_vlans

You didn't think we were done did you?

For the last step here, we'll do the exact opposite to see if all of the vlans that are actually present on the device are also defined in the vlans.yaml file on github. We want to make sure that no body snuck in and configured a VLAN in our production environment when we weren't looking, right?

In [237]:
for vlan in dev_vlans:
        if vlan in git_vlans:
            print ( "VLAN " + vlan['vlanId'] + " should be there")
        elif vlan not in git_vlans:
            print ( "\nSomebody added VLAN " + vlan['vlanId'] + " when we weren't looking. \n \nGo slap them please.\n\n")

VLAN 1 should be there
VLAN 2 should be there
VLAN 3 should be there

Somebody added VLAN 5 when we weren't looking. 
 
Go slap them please.


VLAN 10 should be there
